In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

import torch

In [2]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.bfloat16,
                                         bnb_4bit_use_double_quant=True,
                                         bnb_4bit_quant_type= "nf4"
                                         )

In [3]:
model = AutoModelForCausalLM.from_pretrained(
	model_name, device_map="auto", torch_dtype=torch.bfloat16, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500, pad_token_id = tokenizer.eos_token_id)

In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a great mentor."),
    ("user", "{input}")
])

In [6]:
output_parser = StrOutputParser()
hf = HuggingFacePipeline(pipeline=pipe)

In [7]:
chain = prompt | hf | output_parser

In [9]:
result = chain.invoke({"input": "how can i improve my software engineering skills?"})
print(result)

System: You are a great mentor.
Human: how can i improve my software engineering skills? 
System: Let's break it down. Here are some suggestions:

1. **Practice coding**: Regularly practice coding in your favorite programming language. Try solving problems on platforms like LeetCode, HackerRank, or CodeWars. 
2. **Learn by doing**: Work on real-world projects, either individually or in teams. This will help you apply theoretical concepts to practical problems.
3. **Read books and articles**: Stay updated with the latest trends and technologies by reading books and articles on software engineering, design patterns, and best practices.
4. **Participate in coding communities**: Join online communities like GitHub, Stack Overflow, or Reddit's r/learnprogramming and r/webdev. These platforms offer valuable resources, feedback, and connections with other developers.
5. **Take online courses**: Websites like Coursera, Udemy, and edX offer courses on software engineering, computer science, and